In [2]:
import pandas as pd
import re

In [115]:
data = pd.read_csv("datasets/spacy_ds.csv", delimiter=';')

In [125]:
train_data = []

In [126]:
for _, row in data.iterrows():
    text = row['text']
    origin = row['origin']
    destination = row['destination']
    detours = row['detours'] if pd.notna(row['detours']) else ""
    
    origin_match = re.search(rf"\b{re.escape(origin)}\b", text.lower())
    if origin_match:
        start_origin = origin_match.start()
        end_origin = origin_match.end()

    destination_match = re.search(rf"\b{re.escape(destination)}\b", text.lower())
    if destination_match:
        start_destination = destination_match.start()
        end_destination = destination_match.end()

    detour_positions = []
    
    if detours:
        for detour in detours.split(","):
            detour = detour.strip()
            detour_match = re.search(rf"\b{re.escape(detour)}\b", text.lower())
            if detour_match:
                start_detour = text.lower().find(detour)
                end_detour = start_detour + len(detour)
                
                if start_detour >= 0 and (
                    end_detour <= start_origin or start_detour >= end_origin
                ) and (
                    end_detour <= start_destination or start_detour >= end_destination
                ):
                    overlap = any(
                        (start < end_detour and end > start_detour)
                        for start, end, _ in detour_positions
                    )
                    if not overlap:
                        detour_positions.append((start_detour, end_detour, "DETOUR"))
    
    entities = [
        (start_origin, end_origin, "ORIGIN"),
        (start_destination, end_destination, "DESTINATION")
    ] + detour_positions

    train_data.append((text, {"entities": entities}))

In [127]:
len(train_data)

10500

In [4]:
import spacy
from spacy.training.example import Example
import random

/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [128]:
nlp = spacy.blank("fr")

if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

ner.add_label("ORIGIN")
ner.add_label("DESTINATION")
ner.add_label("DETOUR")

optimizer = nlp.begin_training()

In [129]:
for itn in range(10):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        try:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], losses=losses, drop=0.5, sgd=optimizer)
        except ValueError as error:
            print(f'Error for text: {text}, annotations: {annotations}')
    print(f"Itération {itn} - Losses: {losses}")

Itération 0 - Losses: {'ner': 20593.038040185715}
Itération 1 - Losses: {'ner': 12278.376547064434}
Itération 2 - Losses: {'ner': 10244.39264769106}
Itération 3 - Losses: {'ner': 8949.409452502552}
Itération 4 - Losses: {'ner': 7993.648581530898}
Itération 5 - Losses: {'ner': 7254.491256480974}
Itération 6 - Losses: {'ner': 6700.574295276065}
Itération 7 - Losses: {'ner': 6609.825125060915}
Itération 8 - Losses: {'ner': 6387.979754652851}
Itération 9 - Losses: {'ner': 6284.619403910409}


In [137]:
nlp.to_disk("models/spacy_model")

In [5]:
nlp = spacy.load('models/spacy_model')

In [6]:
from langdetect import detect, LangDetectException

In [8]:
from unidecode import unidecode
import re

In [9]:
def test_trip_request(text):
    text = unidecode(text).lower()
    text = re.sub(r'[^\w\s]', '', text)

    doc = nlp(text)
    origin, destination = None, None
    detours = []

    for ent in doc.ents:
        if ent.label_ == "ORIGIN":
            origin = ent.text
        elif ent.label_ == "DESTINATION":
            destination = ent.text
        elif ent.label_ == "DETOUR":
            detours.append(ent.text)
    
    if origin and destination:
        return (text, origin, destination, detours)
    else:
        return "NOT_TRIP"

In [10]:
def show_infos(origin, destination, detours):
    print(f'Depart: {origin}')
    print(f'Arrivée: {destination}')
    detours_sentence = ""
    for i in range(len(detours)):
        if(i == len(detours) - 1):
            detours_sentence += detours[i]
        else:
            detours_sentence += detours[i] + ", "
    print(f'Détours: {detours_sentence if len(detours) > 0 else "Aucun"}\n')

In [13]:
val_ds = pd.read_csv("datasets/spacy_ds_val.csv", delimiter=';')

In [11]:
test_phrases = [
    "Je veux aller de paris à vendenheim",
    "J'aimerai aller de lille à nice",
    "Voyage de rouen jusqu'à nice",
    "Quel est le trajet de toulouse à bordeaux ?",
    "Je veux aller de paris à lyon en passant par nice",
    "En passant par toulouse, je veux aller de paris à lyon",
    "Quel est le trajet de strasbourg à bordeaux ?",
    "Quel est le trajet de bordeaux à strasbourg en passant par lyon ?",
    "Comment me rendre à strasbourg depuis nice ?",
    "En passant par Lyon, j'aimerai aller à Nice depuis Strasbourg",
]

In [15]:
result = 0
not_trip = []
for _, row in val_ds.iterrows():
    value = test_trip_request(row['text'])
    if value != 'NOT_TRIP':
        result += 1
    else:
        print(row['text'])
        not_trip.append(row['text'])

je suis a st martin de blagny et jaimerais me rendre a gurcy le chatel en passant par lourdoueix st michel bazus montfort l amaury et st brieuc des iffs


In [16]:
accuracy = (result * 100) / len(val_ds)
print(f'accuracy: {accuracy:.02f}%')

accuracy: 93.33%


In [17]:
not_trip

['je suis a st martin de blagny et jaimerais me rendre a gurcy le chatel en passant par lourdoueix st michel bazus montfort l amaury et st brieuc des iffs']

In [12]:
for phrase in test_phrases:
    print(f"sentence: {phrase}")
    print(test_trip_request(phrase.lower()))

sentence: Je veux aller de paris à vendenheim
('je veux aller de paris a vendenheim', 'paris', 'vendenheim', [])
sentence: J'aimerai aller de lille à nice
('jaimerai aller de lille a nice', 'lille', 'nice', [])
sentence: Voyage de rouen jusqu'à nice
('voyage de rouen jusqua nice', 'rouen', 'nice', [])
sentence: Quel est le trajet de toulouse à bordeaux ?
('quel est le trajet de toulouse a bordeaux ', 'toulouse', 'bordeaux', [])
sentence: Je veux aller de paris à lyon en passant par nice
('je veux aller de paris a lyon en passant par nice', 'paris', 'lyon', ['nice'])
sentence: En passant par toulouse, je veux aller de paris à lyon
('en passant par toulouse je veux aller de paris a lyon', 'paris', 'lyon', ['toulouse'])
sentence: Quel est le trajet de strasbourg à bordeaux ?
('quel est le trajet de strasbourg a bordeaux ', 'strasbourg', 'bordeaux', [])
sentence: Quel est le trajet de bordeaux à strasbourg en passant par lyon ?
('quel est le trajet de bordeaux a strasbourg en passant par l

In [135]:
phrase = "j'aimerais me rendre à lyon depuis strasbourg"
test_trip_request(phrase)

('jaimerais me rendre a lyon depuis strasbourg', 'strasbourg', 'lyon', [])